In [289]:
import pandas as pd
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import decomposition
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt

In [290]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_result = df_test[['PassengerId']].copy()
df_result['Survived'] = pd.Series(np.random.randn(len(df_result)), index=df_result.index)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [291]:
"""fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Survived?')
ax.set_ylabel('No of people')
ax.set_title("Total survived vs not survived")
pd.value_counts(df_train['Survived']).plot.bar()
#df_train['Survived'].plot(kind='bar')
plt.show()"""

'fig = plt.figure()\nax = fig.add_subplot(1,1,1)\nax.set_xlabel(\'Survived?\')\nax.set_ylabel(\'No of people\')\nax.set_title("Total survived vs not survived")\npd.value_counts(df_train[\'Survived\']).plot.bar()\n#df_train[\'Survived\'].plot(kind=\'bar\')\nplt.show()'

In [292]:
def normalizeDf(df):
    temp = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    temp_scaled = min_max_scaler.fit_transform(temp)
    df = pd.DataFrame(temp_scaled)
    df.columns = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
    return df

In [293]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
fare_mean = df_train['Fare'].mean()
fare_median = df_train['Fare'].median()

for i in range(0,len(df_train['Fare'])):
    val = df_train.iloc[i]['Fare']
    if(val<=fare_median):
        df_train.at[i,'Fare']  = 1
    elif(val>fare_median and val<=fare_mean):
        df_train.at[i,'Fare'] = 2
    else:
        df_train.at[i,'Fare'] = 3

for i in range(0, len(df_test['Fare'])):
    val = df_test.iloc[i]['Fare']
    if(val<=fare_median):
        df_test.at[i,'Fare'] = 1
    elif(val>fare_median and val<=fare_mean):
        df_test.at[i,'Fare'] = 2
    else:
        df_test.at[i,'Fare'] = 3

df_train['Sex'].replace(['female','male'],[1,2],inplace=True)
df_test['Sex'].replace(['female','male'],[1,2],inplace=True)
df_train['Embarked'].replace(['Q','S','C'],[1,2,3],inplace=True)
df_test['Embarked'].replace(['Q','S','C'],[1,2,3],inplace=True)

df_train_mod = normalizeDf(df_train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']])
df_test_mod = normalizeDf(df_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']])
#df_test['Pclass'] = (eg_array - eg_array.mean(axis=0)) / eg_array.std(axis=0)

X = df_train_mod[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y = df_train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

#X_train = df_train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
#Y_train = df_train['Survived']

X_toTest = df_test_mod[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [294]:
X_train.head(6)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
6,0.0,1.0,0.6750,0.000,0.000000,1.0,0.666667
718,1.0,1.0,0.0000,0.000,0.000000,0.5,0.333333
685,0.5,1.0,0.3125,0.125,0.333333,1.0,1.000000
73,1.0,1.0,0.3250,0.125,0.000000,0.0,1.000000
882,1.0,0.0,0.2750,0.000,0.000000,0.0,0.666667
328,1.0,0.0,0.3875,0.125,0.166667,0.5,0.666667


In [295]:
pca = decomposition.PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
X_toTest = pca.transform(X_toTest)

In [296]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [100,10,1,0.1,1e-2,1e-3,1e-4],
                     'C': [0.1,1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}]
clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5)

In [297]:
#clf = svm.SVC(kernel='linear')
#clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(clf.best_params_)

{'kernel': 'rbf', 'C': 1, 'gamma': 10}


In [298]:
y_pred = clf.predict(X_toTest)
"""y_pred = clf.predict(X_test)
acc = accuracy_score(y_test,y_pred)
acc"""

'y_pred = clf.predict(X_test)\nacc = accuracy_score(y_test,y_pred)\nacc'

In [299]:
df_result['Survived'] = y_pred
df_result.to_csv('submission.csv')